In [ ]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import urllib.request
import time
import requests
import re
import csv
import os
import sys
import json
import warnings
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
 
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
 
cred = credentials.Certificate('picable-9238b-firebase-adminsdk-7603p-ca77917236.json')
firebase_admin.initialize_app(cred,{
    'databaseURL' : 'https://picable-9238b-default-rtdb.firebaseio.com/'
})
 
seoul = []
 
gu = open('C://Users//윤종문//Desktop//gu.txt','r', encoding='UTF8')
while True:
    line = gu.readline().strip()
    if not line: break
    seoul.append(line)
 
driver = webdriver.Chrome('C://Users//윤종문//Desktop//chromedriver')
driver.implicitly_wait(3)
naver_url = "https://map.naver.com/v5/"
driver.get(naver_url)
soup = BeautifulSoup(urllib.request.urlopen(naver_url).read(), "html.parser")
 
print('[*] Start Crawling...')
 
 
def search():
    driver.switch_to.default_content()
    time.sleep(1)
        
    frame = driver.find_element(By.ID, 'searchIframe') #iframe 태그 엘리먼트 찾기
    driver.switch_to.frame(frame) #프레임 이동
    time.sleep(1)
 
    storelist = driver.find_element(By.CSS_SELECTOR, "div._1Az1K")
    driver.execute_script("arguments[0].scrollBy(0, 4320)", storelist)
    time.sleep(1)
    
    stores_box = driver.find_element(By.XPATH, "/html/body/div[3]/div/div[2]/div[1]/ul")
    stores = driver.find_elements(By.CSS_SELECTOR, "li._1EKsQ._12tNp")
 
    for store in stores: #한 페이지 내에서의 반복문. 순차적으로 가게 정보에 접근
        count = 1
        driver.switch_to.default_content()
        time.sleep(1)
        
        frame = driver.find_element(By.ID, 'searchIframe') #iframe 태그 엘리먼트 찾기
        driver.switch_to.frame(frame) #프레임 이동
        time.sleep(1)
 
        #이름
        try:
            name = store.find_element(By.CLASS_NAME, 'place_bluelink').text
            print(count + ': ' + name)
        except:
            name = ''
            #print('no name')
 
        img = []
        try:#사진
            imgs = store.find_elements(By.CSS_SELECTOR, 'div.place_thumb')
            for i in imgs:
                tmp0 = i.find_element(By.CLASS_NAME, 'cb7hz').get_attribute('style')
                tmp1 = tmp0.split('url("')[1].split('");')[0]
                img.append(tmp1)
            #print(img)
        except:
            #print('no image')
            continue
 
        click_name = store.find_element(By.CSS_SELECTOR, "div._1sfuL")
        click_name.click()
        # 가게 주소, 홈페이지 링크를 확인하려면 가게 이름을 클릭해 세부 정보를 띄워야 함.
 
        driver.switch_to.default_content()
        time.sleep(1)
       
        frame = driver.find_element(By.ID, 'entryIframe') #iframe 태그 엘리먼트 찾기
        driver.switch_to.frame(frame) #프레임 이동
        time.sleep(1)
 
        '''tag = []
        try: #태그
            tagDiv = driver.find_element(By.CSS_SELECTOR, "place_section._35XFB")
            idx = 0
        
            tags = tagDiv.find_elements(By.CSS_SELECTOR, "span._2irYJ")
            for i in tags:
                if idx == 0:
                    tmp = i.text
                    idx = idx + 1
                else:
                    tmp0 = i.text
                    tmp = tmp0.split(', ')[1]
                tag.append(tmp)
            print(tag)
        except:
            print('no tag')'''
 
        try:#카테고리
            cate = address = driver.find_element(By.CSS_SELECTOR, "span._3ocDE").text
            #print(cate)
        except:
            cate = ''
            #print('no category')
            
        try:#위치
            address = driver.find_element(By.CSS_SELECTOR, "span._2yqUQ").text
            #print(address)
        except:
            address = ''
            #print('no address')
 
        try:#별점
            rating = driver.find_element(By.CSS_SELECTOR, "span._1Y6hi._1A8_M > em").text
            #print(rating)
        except:
            rating = ''
            #print('no rating')
 
        try:#링크
            link_url = driver.find_element(By.CSS_SELECTOR, 'a._1RUzg').text
            #print(link_url)
        except:
            link_url = ''
            #print('no link')
 
        try:#번호
            tel = driver.find_element(By.CSS_SELECTOR, 'span._3ZA0S').text
            #print(tel)
        except:
            tel = ''
            #print('no tel')
 
        hour = {}
        try:#시간
            click_name = driver.find_element(By.CSS_SELECTOR, "div._20Y9l")
            click_name.click()
            time.sleep(2)
 
            '''hours = driver.find_elements(By.CSS_SELECTOR, 'span._20pEw')
            for i in hours:
                one = i.find_element(By.CSS_SELECTOR, 'span._1v6gO').text
                two = i.find_element(By.CSS_SELECTOR, 'div._3uEtO').text
                hour[one] = two'''
 
            one = driver.find_elements(By.CSS_SELECTOR, 'span._1v6gO')
            two = driver.find_elements(By.CSS_SELECTOR, 'div._3uEtO')
 
            for o,t in zip(one,two):
                hour[o.text] = t.text             
            #print(hour)
        except:
            continue
            #print('no time')
 
        try:#리뷰
            reviewTab = driver.find_element(By.XPATH, '//span[contains(text(), "리뷰") and @class="_3aXen"]')
            reviewTab.click()
            time.sleep(2)
 
            strReview = driver.find_element(By.CLASS_NAME, 'place_section_count').text
 
            try:
                strReview = strReview.replace(',', '')
            except:
                #print("it's okay")
                continue
            floatReview = float(strReview)
            intReview = int(floatReview)
 
            if intReview > 10:
                click_name = driver.find_element(By.CSS_SELECTOR, "a._3iTUo")
                click_name.click()
                time.sleep(1)
        
            review = {}
            reviews = driver.find_elements(By.CSS_SELECTOR, "li._3l2Wz")
            idx = 0
        
            for i in reviews:
                r = []
                
                try:
                    click_name = i.find_element(By.CSS_SELECTOR, "span._3_09q")
                    click_name.click()
                    time.sleep(1)
                except:
                    #print('no extend')
                    continue
                finally:
                    textReview = i.find_element(By.CSS_SELECTOR, 'span.WoYOw').text
                r.append(textReview)
                
                try:
                    imgReviews = i.find_elements(By.CSS_SELECTOR, "div._2IOo9")
                    for j in imgReviews:
                        tmp0 = j.find_element(By.CSS_SELECTOR, "div.cb7hz").get_attribute('style')
                        tmp1 = tmp0.split('url("')[1].split('");')[0]
                        r.append(tmp1)
                except:
                    #print('no img')
                    continue
                review[idx] = r
                #print(idx)
                idx = idx + 1
            #print(review)
        except:
            #print('no review')
            continue
 
        ref = db.reference(name)
        ref.update({'category' : cate})
        ref.update({'map' : address})
        ref.update({'image' : img})
        ref.update({'score' : rating})
        ref.update({'time' : hour})
        ref.update({'tel' : tel})
        ref.update({'review' : review})
        ref.update({'link' : link_url}) 
 
for i in seoul:
    print("---- " + i + " ----")
 
    query = i + u' 맛집'
 
    searchBox = driver.find_element(By.CSS_SELECTOR, "div.input_box>input.input_search")
    searchBox.clear()
    searchBox.send_keys(query)
    time.sleep(1)
    searchBox.send_keys(Keys.ENTER)
    time.sleep(2)
    
    search()
 
    # 다음 페이지로 이동
    for i in range(1):
        try:
            driver.switch_to.default_content()
            time.sleep(1)
        
            frame = driver.find_element(By.ID, 'searchIframe') #iframe 태그 엘리먼트 찾기
            driver.switch_to.frame(frame) #프레임 이동
            time.sleep(1)
 
            tmp = driver.find_elements(By.CLASS_NAME, '_3Dl4U')
            next_page = tmp[1]
        
            next_page.click()
            time.sleep(2)
        
            #print('다음 페이지')
            
        except:
            #print('다음 페이지 없음')
            continue
 
        search()  
    
    driver.switch_to.default_content()
 
# 크롭 웹페이지를 닫음
driver.close()
print('******************** 크롤링 완료 ********************')